In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_syllable_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', other_features=True)

# syllable_letters_translator = create_syllable_letters_translator(max_syllable, syllable_dictionary, dictionary, vowels)

data = Data('sl', accent_classification=True)
data.generate_data('syllabled_letters_accent_classification_train',
                   'syllabled_letters_accent_classification_test',
                   'syllabled_letters_accent_classification_validate', force_override=False)

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [4]:
data.x_train = np.concatenate((data.x_train, data.x_test), axis=0)
data.x_other_features_train = np.concatenate((data.x_other_features_train, data.x_other_features_test), axis=0)
data.y_train = np.concatenate((data.y_train, data.y_test), axis=0)
y_array_train = np.asarray(data.y_train)
accentuation_length_train = (y_array_train > 0).sum()
y_array_test = np.asarray(data.y_test)
accentuation_length_test = (y_array_test > 0).sum()

In [13]:
print(len(data.x_train))
print(len(data.y_train))

print(accentuation_length_test)

482209
482209
54846


In [14]:
num_examples = len(data.x_train) # training set size

nn_output_dim = 13
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20



conv_input_shape=(10, 252)
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# x_conv = Conv1D(43, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [15]:

history = model.fit_generator(data.generator('train', batch_size), 
                              accentuation_length_train/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch)

Epoch 1/400
1625/1624 [==============================] - 21s - loss: 0.2161 - actual_accuracy: 0.1490    
Epoch 2/400
1625/1624 [==============================] - 22s - loss: 0.1193 - actual_accuracy: 0.5458    
Epoch 3/400
1625/1624 [==============================] - 24s - loss: 0.0877 - actual_accuracy: 0.6886    
Epoch 4/400
1625/1624 [==============================] - 28s - loss: 0.0680 - actual_accuracy: 0.7723    
Epoch 5/400
1625/1624 [==============================] - 35s - loss: 0.0521 - actual_accuracy: 0.8323    
Epoch 6/400
1625/1624 [==============================] - 33s - loss: 0.0430 - actual_accuracy: 0.8630    
Epoch 7/400
1625/1624 [==============================] - 33s - loss: 0.0359 - actual_accuracy: 0.8867    
Epoch 8/400
1625/1624 [==============================] - 37s - loss: 0.0326 - actual_accuracy: 0.8951    
Epoch 9/400
1625/1624 [==============================] - 38s - loss: 0.0279 - actual_accuracy: 0.9099    
Epoch 10/400
1625/1624 [======================

1625/1624 [==============================] - 34s - loss: 0.0034 - actual_accuracy: 0.9900    
Epoch 75/400
1625/1624 [==============================] - 34s - loss: 0.0032 - actual_accuracy: 0.9910    
Epoch 76/400
1625/1624 [==============================] - 35s - loss: 0.0032 - actual_accuracy: 0.9913    
Epoch 77/400
1625/1624 [==============================] - 35s - loss: 0.0034 - actual_accuracy: 0.9899    - ETA: 2s - los
Epoch 78/400
1625/1624 [==============================] - 34s - loss: 0.0037 - actual_accuracy: 0.9892    
Epoch 79/400
1625/1624 [==============================] - 38s - loss: 0.0033 - actual_accuracy: 0.9910    - ETA: 3s - loss - ETA: 1s - loss: 0.0033 - actual
Epoch 80/400
1625/1624 [==============================] - 37s - loss: 0.0024 - actual_accuracy: 0.9934    - ETA:
Epoch 81/400
1625/1624 [==============================] - 35s - loss: 0.0039 - actual_accuracy: 0.9893    
Epoch 82/400
1625/1624 [==============================] - 35s - loss: 0.0031 - actual_

1625/1624 [==============================] - 27s - loss: 0.0020 - actual_accuracy: 0.9950    
Epoch 148/400
1625/1624 [==============================] - 34s - loss: 0.0021 - actual_accuracy: 0.9941    
Epoch 149/400
1625/1624 [==============================] - 36s - loss: 0.0018 - actual_accuracy: 0.9953    
Epoch 150/400
1625/1624 [==============================] - 36s - loss: 0.0021 - actual_accuracy: 0.9945    
Epoch 151/400
1625/1624 [==============================] - 35s - loss: 0.0018 - actual_accuracy: 0.9950    
Epoch 152/400
1625/1624 [==============================] - 33s - loss: 0.0020 - actual_accuracy: 0.9943    
Epoch 153/400
1625/1624 [==============================] - 33s - loss: 0.0020 - actual_accuracy: 0.9946    
Epoch 154/400
1625/1624 [==============================] - 33s - loss: 0.0020 - actual_accuracy: 0.9947    
Epoch 155/400
1625/1624 [==============================] - 31s - loss: 0.0019 - actual_accuracy: 0.9949    
Epoch 156/400
1625/1624 [=================

1625/1624 [==============================] - 36s - loss: 0.0013 - actual_accuracy: 0.9968    
Epoch 221/400
1625/1624 [==============================] - 39s - loss: 0.0018 - actual_accuracy: 0.9953    
Epoch 222/400
1625/1624 [==============================] - 34s - loss: 0.0015 - actual_accuracy: 0.9967    
Epoch 223/400
1625/1624 [==============================] - 35s - loss: 0.0017 - actual_accuracy: 0.9953    
Epoch 224/400
1625/1624 [==============================] - 34s - loss: 0.0017 - actual_accuracy: 0.9960    
Epoch 225/400
1625/1624 [==============================] - 35s - loss: 0.0015 - actual_accuracy: 0.9965    
Epoch 226/400
1625/1624 [==============================] - 35s - loss: 0.0018 - actual_accuracy: 0.9955    - ETA: 3s - ETA: 1s - loss: 0.0018 
Epoch 227/400
1625/1624 [==============================] - 30s - loss: 0.0015 - actual_accuracy: 0.9961    
Epoch 228/400
1625/1624 [==============================] - 27s - loss: 0.0016 - actual_accuracy: 0.9958    
Epoch 2

1625/1624 [==============================] - 37s - loss: 0.0012 - actual_accuracy: 0.9967    
Epoch 295/400
1625/1624 [==============================] - 40s - loss: 0.0014 - actual_accuracy: 0.9966    
Epoch 296/400
1625/1624 [==============================] - 39s - loss: 0.0011 - actual_accuracy: 0.9972    
Epoch 297/400
1625/1624 [==============================] - 39s - loss: 0.0013 - actual_accuracy: 0.9971    
Epoch 298/400
1625/1624 [==============================] - 39s - loss: 0.0017 - actual_accuracy: 0.9954    - ETA: 2s - loss: - ETA: 0s - loss: 0.0017 - actual_ac
Epoch 299/400
1625/1624 [==============================] - 41s - loss: 0.0016 - actual_accuracy: 0.9963    
Epoch 300/400
1625/1624 [==============================] - 40s - loss: 9.9441e-04 - actual_accuracy: 0.9973    - - ETA: 2s - loss: 9.93
Epoch 301/400
1625/1624 [==============================] - 39s - loss: 0.0014 - actual_accuracy: 0.9964    
Epoch 302/400
1625/1624 [==============================] - 37s - los

1625/1624 [==============================] - 42s - loss: 0.0016 - actual_accuracy: 0.9957    
Epoch 368/400
1625/1624 [==============================] - 37s - loss: 0.0014 - actual_accuracy: 0.9968    
Epoch 369/400
1625/1624 [==============================] - 38s - loss: 0.0011 - actual_accuracy: 0.9966    
Epoch 370/400
1625/1624 [==============================] - 38s - loss: 0.0012 - actual_accuracy: 0.9970    
Epoch 371/400
1625/1624 [==============================] - 37s - loss: 0.0011 - actual_accuracy: 0.9971    
Epoch 372/400
1625/1624 [==============================] - 37s - loss: 0.0013 - actual_accuracy: 0.9970    
Epoch 373/400
1625/1624 [==============================] - 37s - loss: 0.0014 - actual_accuracy: 0.9964    
Epoch 374/400
1625/1624 [==============================] - 39s - loss: 0.0013 - actual_accuracy: 0.9964    
Epoch 375/400
1625/1624 [==============================] - 44s - loss: 0.0012 - actual_accuracy: 0.9967    
Epoch 376/400
1625/1624 [=================

In [16]:
name = '20_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [15]:
# dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
# train_content, test_content, validate_content = split_content(content, 0.2, '../../internal_representations/inputs/content_shuffle_vector.h5')
# feature_dictionary = create_feature_dictionary()
# syllable_dictionary = create_syllables_dictionary(content, vowels)
# max_syllable = get_max_syllable(syllable_dictionary)


content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)

In [8]:
%run ../../../prepare_data.py
# generate_X_and_y(dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels, feature_dictionary)